In [20]:
import cv2
import numpy as np

In [ ]:
class Frame(object):
    def __init__(self,img) -> None:
        self.frame=img
        

In [21]:
def Affine_trans(prev,curr):
    prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    curr_gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)


    prev_pts = cv2.goodFeaturesToTrack(prev_gray,
                                       maxCorners=200,
                                       qualityLevel=0.01,
                                       minDistance=30,
                                       blockSize=3)
    
    curr_pts, status, err = cv2.calcOpticalFlowPyrLK(
        prev_gray, curr_gray, prev_pts, None)
    
    assert prev_pts.shape == curr_pts.shape

    # Filter only valid points
    idx = np.where(status == 1)[0]
    prev_pts = prev_pts[idx]
    curr_pts = curr_pts[idx]

    #Find transformation matrix

    [m, inliers] = cv2.estimateAffinePartial2D(prev_pts, curr_pts)
    M_matrix = np.vstack((m, np.array([0, 0, 1])))

    return M_matrix
    
    

    

In [38]:
import numpy as np
import cv2
 
# Read input video
cap = cv2.VideoCapture('video3.mp4')

if (cap.isOpened() == False):
    print("Error reading video file")
 
frames=[]


# Get frame count
fps = int(cap.get(cv2.CAP_PROP_FPS))

print(fps)
 
# Get width and height of video stream
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
 
# Define the codec for output video
fourcc = cv2.VideoWriter_fourcc(*'MJPG')


 
# Set up output video
out = cv2.VideoWriter('video3_out.avi', fourcc, fps, (w, h))


while(True):
    ret, img = cap.read()

    if not ret:
       break
    
    frames.append(img)
    
    if len(frames)==1:
       out.write(frames[-1])
       continue

    T=Affine_trans(frames[-2],frames[-1])


    inv_T=np.linalg.inv(T)

    frames[-1] = cv2.warpAffine(frames[-1], inv_T[:2, :3], (w, h))

    # print("Frame: " + str(i) + "/" + str(n_frames) +
    #     " -  Tracked points : " + str(len(prev_pts)))



    cv2.waitKey(30)
    out.write(frames[-1])
    cv2.imshow("frame",frames[-1])
    cv2.imshow("original", img)

cap.release()
out.release()
cv2.destroyAllWindows()


Error reading video file
0


[mov,mp4,m4a,3gp,3g2,mj2 @ 0x1f77c00] error reading header
[ERROR:0@5889.974] global cap.cpp:643 open VIDEOIO(CV_MJPEG): raised OpenCV exception:

OpenCV(4.8.1) /io/opencv/modules/videoio/src/cap_mjpeg_encoder.cpp:436: error: (-215:Assertion failed) fps >= 1 in function 'open'


